**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [12]:
!pip install keras-flops

In [13]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [14]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 0.01

In [15]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [16]:
# Преобразуйте метки классов в one_hot формат
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

In [17]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [18]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(32, 32, 3)),

    tf.keras.layers.SeparableConv2D(50, (3, 3), strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.SeparableConv2D(50, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.1),

    tf.keras.layers.Dense(CLASSES, activation='sigmoid')
])

In [19]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
FLOPs: 0.7010e6

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math b

2023-01-25 12:12:43.154751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 12:12:43.155587: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 12:12:43.155734: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 12:12:43.156298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 12:12:43.157162: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 12:12:43.157939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [20]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 32, 32, 3)         7         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 16, 16, 50)        227       
_________________________________________________________________
batch_normalization_3 (Batch (None, 16, 16, 50)        200       
_________________________________________________________________
activation_3 (Activation)    (None, 16, 16, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 6, 6, 50)          3000      
_________________________________________________________________
batch_normalization_4 (Batch (None, 6, 6, 50)         

In [21]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [22]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=256
)

Epoch 1/256
391/391 [==============================] - 4s 8ms/step - loss: 3.4691 - accuracy: 0.1790 - val_loss: 3.7637 - val_accuracy: 0.1584
Epoch 2/256
391/391 [==============================] - 3s 8ms/step - loss: 2.9104 - accuracy: 0.2774 - val_loss: 3.0504 - val_accuracy: 0.2618
Epoch 3/256
391/391 [==============================] - 3s 8ms/step - loss: 2.6632 - accuracy: 0.3286 - val_loss: 2.7076 - val_accuracy: 0.3255
Epoch 4/256
391/391 [==============================] - 3s 8ms/step - loss: 2.5027 - accuracy: 0.3592 - val_loss: 2.6553 - val_accuracy: 0.3377
Epoch 5/256
391/391 [==============================] - 3s 8ms/step - loss: 2.3704 - accuracy: 0.3885 - val_loss: 2.5728 - val_accuracy: 0.3517
Epoch 6/256
391/391 [==============================] - 3s 8ms/step - loss: 2.2794 - accuracy: 0.4060 - val_loss: 2.4447 - val_accuracy: 0.3803
Epoch 7/256
391/391 [==============================] - 3s 8ms/step - loss: 2.1937 - accuracy: 0.4235 - val_loss: 2.4367 - val_accuracy: 0.3831

In [27]:
score=model.evaluate(X_val,y_val)
print(score)

313/313 [==============================] - 1s 3ms/step - loss: 2.3570 - accuracy: 0.4046
[2.356990098953247, 0.40459999442100525]


In [29]:
model.save(filepath=f"accuracy:{score[1]:.2f}.hdf5")